<a href="https://colab.research.google.com/github/TutuBraz/avocado-predict-price/blob/main/avocado_price_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo: Criar um modelo que consiga prever o preço do abacate.

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_avocado = pd.read_csv('//content/Avocado.csv')
df_avocado

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27318,2015-11-22,1.97,6249.43,1733.40,2873.92,30.95,1611.16,1590.00,21.16,0.0,organic,2015,WestTexNewMexico
27319,2015-11-29,2.08,4638.10,1395.02,2238.04,61.71,943.33,943.33,0.00,0.0,organic,2015,WestTexNewMexico
27320,2015-12-13,1.80,7836.65,2194.49,2981.01,25.97,2635.18,2598.45,36.73,0.0,organic,2015,WestTexNewMexico
27321,2015-12-20,1.92,6255.19,1512.45,2407.32,11.78,2323.64,2213.72,109.92,0.0,organic,2015,WestTexNewMexico


O df está empilhado pelas regiões

# Data cleaning

In [3]:
df_avocado.region.unique()

array(['Albany', 'Atlanta', 'Baltimore/Washington', 'BaltimoreWashington',
       'Boise', 'Boston', 'Buffalo/Rochester', 'BuffaloRochester',
       'California', 'Charlotte', 'Chicago', 'Cincinnati/Dayton',
       'CincinnatiDayton', 'Columbus', 'Dallas/Ft. Worth',
       'DallasFtWorth', 'Denver', 'Detroit', 'Grand Rapids',
       'GrandRapids', 'Great Lakes', 'GreatLakes', 'Harrisburg/Scranton',
       'HarrisburgScranton', 'Hartford/Springfield',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'Las Vegas', 'LasVegas', 'Los Angeles', 'LosAngeles', 'Louisville',
       'Miami/Ft. Lauderdale', 'MiamiFtLauderdale', 'Midsouth',
       'Nashville', 'New Orleans/Mobile', 'New York', 'NewOrleansMobile',
       'NewYork', 'Northeast', 'Northern New England',
       'NorthernNewEngland', 'Orlando', 'Philadelphia', 'Phoenix/Tucson',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'Raleigh/Greensboro', 'RaleighGreensboro', 'Richmond/Norfolk',


In [4]:
df_avocado.type.unique()

array(['conventional', 'organic'], dtype=object)

As regiões aparentemente as regioes estão com valores duplicados. Ex: West Tex/New Mexico e WestTexNewMexico

In [5]:
df_avocado.duplicated().sum()

2

In [6]:
df_avocado.groupby('region').count()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year
region,,,,,,,,,,,,
Albany,506,506,506,506,506,506,506,506,506,506,506,506
Atlanta,506,506,506,506,506,506,506,506,506,506,506,506
Baltimore/Washington,402,402,402,402,402,402,402,402,402,402,402,402
BaltimoreWashington,104,104,104,104,104,104,104,104,104,104,104,104
Boise,506,506,506,506,506,506,506,506,506,506,506,506
...,...,...,...,...,...,...,...,...,...,...,...,...
Total U.S.,402,402,402,402,402,402,402,402,402,402,402,402
TotalUS,104,104,104,104,104,104,104,104,104,104,104,104
West,506,506,506,506,506,506,506,506,506,506,506,506


Observo que as regiões não estão duplicadas e sim estão divididas, as regioes tem que somar um total de 506 valores.

In [7]:
#criando lista com os valores invalido
wrong = []
region_list = list(df_avocado.region.unique())
for i in region_list:
  if '/'  in i or ' ' in i or '.'  in i:
    wrong.append(i)

#criando cópia dos valores errado para substituição 
right = wrong[:]

#alterando valores errado para forma correta
for i in range(len(wrong)): 
  for j in [' ','.','/']:
    right[i] = right[i].replace(j, '')
  
#Criando dicionário que relaciona valores incorreto com corretos
valores = {}
for k in range(len(wrong)):
  valores[wrong[k]]=right[k]

#substituindo valores
df_avocado = df_avocado.replace({'region':valores})
     

In [8]:
df_avocado.groupby('region').count()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year
region,,,,,,,,,,,,
Albany,506,506,506,506,506,506,506,506,506,506,506,506
Atlanta,506,506,506,506,506,506,506,506,506,506,506,506
BaltimoreWashington,506,506,506,506,506,506,506,506,506,506,506,506
Boise,506,506,506,506,506,506,506,506,506,506,506,506
Boston,506,506,506,506,506,506,506,506,506,506,506,506
BuffaloRochester,506,506,506,506,506,506,506,506,506,506,506,506
California,506,506,506,506,506,506,506,506,506,506,506,506
Charlotte,506,506,506,506,506,506,506,506,506,506,506,506
Chicago,506,506,506,506,506,506,506,506,506,506,506,506


Problemas ao lidar com data frame:

1.  As PLU não somam o volume total de abacates vendidos.





In [9]:
df_avocado.rename(columns = {'Date': 'date', 'AveragePrice': 'average_price', 'Total Volume': 'total_volume', 'Total Bags': 'total_bags', 'Small Bags': 'small_bags',
                             'Large Bags': "large_bags",'XLarge Bags': "xlarge_bags",'type': 'type'}, inplace = True)

In [10]:
df_avocado

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27318,2015-11-22,1.97,6249.43,1733.40,2873.92,30.95,1611.16,1590.00,21.16,0.0,organic,2015,WestTexNewMexico
27319,2015-11-29,2.08,4638.10,1395.02,2238.04,61.71,943.33,943.33,0.00,0.0,organic,2015,WestTexNewMexico
27320,2015-12-13,1.80,7836.65,2194.49,2981.01,25.97,2635.18,2598.45,36.73,0.0,organic,2015,WestTexNewMexico
27321,2015-12-20,1.92,6255.19,1512.45,2407.32,11.78,2323.64,2213.72,109.92,0.0,organic,2015,WestTexNewMexico


In [11]:
#valores ausentes
df_avocado.isnull().sum()

date             0
average_price    0
total_volume     0
4046             0
4225             0
4770             0
total_bags       0
small_bags       0
large_bags       0
xlarge_bags      0
type             0
year             0
region           0
dtype: int64

In [12]:
df_avocado.duplicated().sum()

2

In [13]:
#Valores duplicados
df_avocado[df_avocado.duplicated()]

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
27132,2017-02-26,1.16,39054.83,3021.26,15568.68,11.77,20453.12,20299.52,153.60,0.0,organic,2017,WestTexNewMexico
27134,2017-03-05,1.23,24969.30,2292.42,4876.69,52.82,17747.37,17114.89,632.48,0.0,organic,2017,WestTexNewMexico


In [14]:
#Removendo duplicatas
df_avocado.drop_duplicates(inplace=True)

#Reorganizando o index
df_avocado.reset_index(drop=True, inplace=True)

In [23]:
df_avocado.reset_index(drop=True, inplace=True)

# EDA

In [17]:
df_avocado.describe(include='all')

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
count,27321,27321.000000,2.732100e+04,2.732100e+04,2.732100e+04,2.732100e+04,2.732100e+04,2.732100e+04,2.732100e+04,27321.000000,27321,27321.00000,27321
unique,253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,54
top,2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conventional,NaN,Albany
freq,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13662,NaN,506
mean,NaN,1.400647,9.136115e+05,2.953402e+05,2.901264e+05,2.216030e+04,3.058950e+05,2.187128e+05,8.203135e+04,5150.764599,NaN,2016.95659,NaN
std,NaN,0.385449,3.702800e+06,1.273054e+06,1.187268e+06,1.031365e+05,1.274895e+06,8.881601e+05,3.917493e+05,30720.300559,NaN,1.40659,NaN
min,NaN,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,NaN,2015.00000,NaN
25%,NaN,1.110000,1.360812e+04,7.963700e+02,2.922520e+03,0.000000e+00,7.703330e+03,5.282770e+03,2.775100e+02,0.000000,NaN,2016.00000,NaN
50%,NaN,1.370000,1.198906e+05,1.004165e+04,2.569837e+04,1.928100e+02,4.776093e+04,3.223936e+04,4.313590e+03,0.000000,NaN,2017.00000,NaN
75%,NaN,1.640000,4.747300e+05,1.133227e+05,1.454874e+05,5.898840e+03,1.461028e+05,1.048429e+05,3.269381e+04,450.740000,NaN,2018.00000,NaN
